Install PyDrive to enable file download and upload

In [0]:
!pip install PyDrive

Importing google specific libraries as well as authenticating the current user

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



Downloading the image training data folder from google drive

In [0]:
download = drive.CreateFile({'id': '1ZrqJtPqAVVuZAycNJN9hl2rEKFiiJ80-'})
download.GetContentFile('frames.tar.7z')

Unzipping img folder

In [0]:
!apt-get install p7zip-full
!p7zip -d frames.tar.7z
!tar -xvf frames.tar!7z e frames.7z

Getting the file from the tar ball

In [0]:
!tar -xvf frames.tar

Process data to be ready for training

In [0]:
import cv2
import numpy as np
import csv
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance


features_directory = './data/'
labels_file = 'driving_log.csv'


def data_loading():
    logs = []
    features = []
    labels = []
    
    #creates datafram from csv file holding angle and image path values
    data = pd.read_csv("driving_log.csv")
    
    #Creates arrays from the angle and image columns
    data_img = np.array(data['Image']) 
    data_img = data_img.reshape(-1, 1)                                                 
    data_angle = np.array(data['Angle'])                                              
    data_angle = data_angle.reshape(-1, 1)
    
    i = 0
    sum_angle = 0
    
    #loops through each image in array
    for j in data_img:
        
        #gets angle and image path from array
        angle = data_angle[i]            
        img_string = str(data_img[i,0])
        img_string = img_string[4:]
        
        #reads image and resizes it from 400x300 to 200x66
        img_array = cv2.imread('frames/' + img_string, 3) 
        img_array = img_array[80:212,0:400]
        img_array = cv2.resize(img_array, (200,66))
        
        #Checks past 10 angles, if average is very small, don't include in final array
        sum_angle += abs(angle)                        
        if sum_angle > 0.15:
          features.append(img_array)
          labels.append(angle)
        if i % 10 == 0:
          sum_angle = 0
        
        #adds additional instances of bigger angles to help make dataset more uniform
        if abs(angle < 0.25) and abs(angle > 0.1):
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
          
        if abs(angle > 0.25):
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle) 
        
        if abs(angle) < 1 and abs(angle) > 0.25:
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
        if abs(angle > 0.5):
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            features.append(cv2.flip(img_array, 1))
            labels.append(angle*(-1))
            features.append(img_array)
            labels.append(angle)
            
        i = i + 1     
    return features, labels
       
#gets final array from data loading function
features, labels = data_loading()

features = np.array(features).astype('float32')
labels = np.array(labels).astype('float32')

#plots a histogram of data to show distribution
plt.hist(labels, bins=20)
plt.show()

#Prints how many data points there are in final array
print(len(features))

#Saves final arrays to pickles
with open("features", "wb") as f:
    pickle.dump(features, f, protocol=4)
with open("labels", "wb") as f:
    pickle.dump(labels, f, protocol=4)
print('done')

Train the model based on final input arrays

In [0]:
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Lambda
from keras.layers import MaxPooling2D, Dropout
from keras.utils import print_summary
import tensorflow as tf
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
import pickle
from keras.optimizers import Adam, SGD
from keras import backend as K

#Function for getting r^2 value
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

#Tier 1 model structure
def keras_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(66,200,3)))
    model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    
    #compile function with mean squared error loss, learning rate of 0.00001 and metric of r^2
    
    model.compile(loss='mse', optimizer=Adam(lr=0.00001), metrics=[coeff_determination])
    
    filepath = "Capstone.h5"
    #makes sure to save the best model only
    checkpoint1 = ModelCheckpoint(filepath, verbose=1, save_best_only=True)
    callbacks_list = [checkpoint1]
    return model, callbacks_list

#loads pickles into arrays
def loadFromPickle():
    with open("features", "rb") as f:
        features = np.array(pickle.load(f))
    with open("labels", "rb") as f:
        labels = np.array(pickle.load(f))
    return features, labels

#flips each image and angle to double dataset
def augmentData(features, labels):
    features = np.append(features, features[:, :, ::-1], axis=0)
    labels = np.append(labels, -labels, axis=0)
    return features, labels


def main():
    
    #load pickles
    features, labels = loadFromPickle()
    
    #Double dataset
    features, labels = augmentData(features, labels)
    
    #Shuffle the data
    
    features, labels = shuffle(features, labels)
    feature = features
    
    #Split data into training and validation sets
    train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0,
                                                        test_size=0.2)
    #reshape inputs arrays
    train_x = train_x.reshape(train_x.shape[0], 66, 200, 3)
    test_x = test_x.reshape(test_x.shape[0], 66, 200, 3)
    
    #Fit model with batch size of 13 for 100 epochs
    model, callbacks_list = keras_model()
    model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=100, batch_size=13,
              callbacks=callbacks_list)
    print_summary(model)
    
    #Save best model
    model.save('Capstone.h5')


main()


Predict angles to see if model is working properly

In [0]:
import numpy as np
import cv2
from keras.models import load_model
import os
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
from google.colab import files
from google.colab import drive
drive.mount('/content/drive/')

#Load previously trained Tier 1 model
model = load_model('Autopilot.h5')

path2= 'frames'
angle = []
Z = []
y = []

#read csv of angles and images into dataframe
data = pd.read_csv("driving_log.csv") 

#Grab individual column data into arrays
data_img = np.array(data['Image']) 
data_img = data_img.reshape(-1, 1)                                                
data_angle = np.array(data['Angle'])

#Normalize angle values
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))              
data_norm = (preprocessing.scale(np.float32(data_angle))/10)

#prediction function
def keras_predict(model, image):
    #Process image
    processed = keras_process_image(image)
    #predict angle based on image
    steering_angle = float(model.predict(processed, batch_size=1))
    return steering_angle

#makes sure image is in correct shape (200x66)
def keras_process_image(img):
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, 66, 200, 3))
    return img

i = 0
sum = 0
smooth_angle = 0

#loops through each image in array
for img in data_img:     
    
    #gets angle and image path
    angle = data_angle[i]            
    img_string = str(data_img[i,0])
    
    #reads, crops, and resizes images to 200x66 from 400x300
    img_string = img_string[4:]
    img_array = cv2.imread('frames/' + img_string, cv2.IMREAD_COLOR)     #creating an array of grayscale imgs
    img_array = img_array[80:212,0:400]
    img_array = cv2.resize(img_array, (200,66))
    
    #add predicted angle to array
    Z.append(keras_predict(model,img_array))
    #add actual angle to array
    y.append(angle)
    i += 1

#combine predicted and actual angle arrays
csv = np.column_stack((Z, y))

#write arrays to csv file
np.savetxt("predict.csv", csv, delimiter=",")
